
- Cuando ya tengamos todos los datos deseados juntaremos los dos csv.
- Para hacer esta unión tendremos que hacer un groupby en la tabla de clima para sacar una media de las medidas climáticas por país.
- Antes de hacer el groupby si nos fijamos tenemos dos columnas rh_profile y wind_profile cuya información es una lista de diccionarios. Si intentamos hacer la media de eso no nos dará un valor real. A este problema ya nos enfrentamos en la clase invertida de ETL-2, donde teníais un Bonus para desempaquetar esta información. En caso de que en aquel ejercicio no lo consigierais os dejamos por aquí una posible solución que nos permite separar esa información en distintas columnas. Os dejamos el código documentado. ⚠️ Os recomendamos que vayáis desgranando el código y viendo lo que nos devuelve cada línea de código para entenderlo mejor.

- Cargaremos los dos ficheros de datos

In [1]:
from IPython.core.interactiveshell import InteractiveShell # Nos permite mostar más de una salida por celda
InteractiveShell.ast_node_interactivity = "all" # Nos permite mostar más de una salida por celda

import requests
import pandas as pd
import numpy as np
df_clima= pd.read_pickle("../Ficheros/ETL1.pkl")
df_attacks= pd.read_csv("../Ficheros/attacks_limpieza_completa.csv", index_col = 0)

In [2]:
pd.set_option('display.max_row', None)
pd.set_option('display.max_columns', None)

- Del dataframe de los ataques nos quedaremos solo con las filas de los países que seleccionamos en la lección de ayer:
    - USA
    - Australia
    - New Zealand
    - South Africa
    - Papua New Guinea

In [3]:
df_seleccion = df_attacks[(df_attacks["country"]== "usa") | (df_attacks["country"] == "australia") | (df_attacks["country"] == "new zealand") | (df_attacks["country"] == "south africa") | (df_attacks["country"] == "papua new guinea")] 

In [4]:
df_seleccion.country.unique()

array(['usa', 'australia', 'south africa', 'new zealand',
       'papua new guinea'], dtype=object)

In [5]:
df_seleccion.sample(2)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,country2,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
700,2013,Unprovoked,australia,26.0,Unespecific,Jan,N,M,-24.776109,134.755,HS,1,0,0,1,0,0,3,5,7,6.191371
4691,1942,Boating,australia,27.0,Unespecific,Unknown,N,M,-24.776109,134.755,HS,1,0,0,1,0,0,3,14,1,6.313925



- Del dataframe de los datos climáticos seleccionaremos todas las columnas.

In [6]:
df_clima['rh_profile'].iloc[0]

[{'layer': '950mb', 'rh': 2},
 {'layer': '900mb', 'rh': 4},
 {'layer': '850mb', 'rh': 5},
 {'layer': '800mb', 'rh': 6},
 {'layer': '750mb', 'rh': 5},
 {'layer': '700mb', 'rh': 7},
 {'layer': '650mb', 'rh': 9},
 {'layer': '600mb', 'rh': 9},
 {'layer': '550mb', 'rh': 4},
 {'layer': '500mb', 'rh': 1},
 {'layer': '450mb', 'rh': 1},
 {'layer': '400mb', 'rh': 1},
 {'layer': '350mb', 'rh': 4},
 {'layer': '300mb', 'rh': 6},
 {'layer': '250mb', 'rh': 1},
 {'layer': '200mb', 'rh': 0}]

In [7]:
x = df_clima['rh_profile'].apply(pd.Series)

for values in range(len(x.columns)):
    nombre='rh' + str(x[values].apply(pd.Series)['layer'][0])
    valores = list(x[values].apply(pd.Series)['rh'])
    df_clima.insert(values, nombre, valores)

In [8]:
df_clima['wind_profile'].iloc[0]

[{'layer': '950mb', 'direction': 5, 'speed': 2},
 {'layer': '900mb', 'direction': 40, 'speed': 2},
 {'layer': '850mb', 'direction': 10, 'speed': 2},
 {'layer': '800mb', 'direction': 285, 'speed': 3},
 {'layer': '750mb', 'direction': 265, 'speed': 4},
 {'layer': '700mb', 'direction': 260, 'speed': 5},
 {'layer': '650mb', 'direction': 245, 'speed': 5},
 {'layer': '600mb', 'direction': 235, 'speed': 5},
 {'layer': '550mb', 'direction': 235, 'speed': 6},
 {'layer': '500mb', 'direction': 235, 'speed': 6},
 {'layer': '450mb', 'direction': 235, 'speed': 5},
 {'layer': '400mb', 'direction': 240, 'speed': 5},
 {'layer': '350mb', 'direction': 245, 'speed': 5},
 {'layer': '300mb', 'direction': 245, 'speed': 5},
 {'layer': '250mb', 'direction': 245, 'speed': 6},
 {'layer': '200mb', 'direction': 245, 'speed': 7}]

In [9]:
y = df_clima['wind_profile'].apply(pd.Series)

for values in range(len(y.columns)):
    nombre='wind_speed' + str(y[values].apply(pd.Series)['layer'][0])
    valores = list(y[values].apply(pd.Series)['speed'])
    df_clima.insert(values, nombre, valores)

In [15]:
def minusculas(x):
    return x.lower()

In [20]:
df_clima['paises'] = df_clima['paises'].apply(minusculas)

In [21]:
df_clima.head(1)

,wind_speed950mb,wind_speed900mb,wind_speed850mb,wind_speed800mb,wind_speed750mb,wind_speed700mb,wind_speed650mb,wind_speed600mb,wind_speed550mb,wind_speed500mb,wind_speed450mb,wind_speed400mb,wind_speed350mb,wind_speed300mb,wind_speed250mb,wind_speed200mb,rh950mb,rh900mb,rh850mb,rh800mb,rh750mb,rh700mb,rh650mb,rh600mb,rh550mb,rh500mb,rh450mb,rh400mb,rh350mb,rh300mb,rh250mb,rh200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud,longitud,paises
0,2,2,2,3,4,5,5,5,6,6,5,5,5,5,6,7,2,4,5,6,5,7,9,9,4,1,1,1,4,6,1,0,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 2}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 5, 'speed': 2...",26,6,3,1009,none,0,0,295,2,39.78373,-100.445882,usa


In [22]:
df_clima_final = df_clima.groupby("paises").mean().reset_index()
df_clima_final

,paises,wind_speed950mb,wind_speed900mb,wind_speed850mb,wind_speed800mb,wind_speed750mb,wind_speed700mb,wind_speed650mb,wind_speed600mb,wind_speed550mb,wind_speed500mb,wind_speed450mb,wind_speed400mb,wind_speed350mb,wind_speed300mb,wind_speed250mb,wind_speed200mb,rh950mb,rh900mb,rh850mb,rh800mb,rh750mb,rh700mb,rh650mb,rh600mb,rh550mb,rh500mb,rh450mb,rh400mb,rh350mb,rh300mb,rh250mb,rh200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.speed,latitud,longitud
0,australia,3.453125,3.187500,3.015625,2.843750,2.812500,2.84375,2.843750,3.171875,3.343750,3.640625,4.265625,4.640625,5.000000,5.390625,5.921875,6.484375,13.718750,11.500000,7.062500,4.140625,2.062500,0.625000,-0.312500,-1.390625,-2.031250,-1.718750,-1.140625,-0.468750,0.328125,2.265625,4.781250,2.562500,97.5,3.515625,-9999.0,-9999.0,-9999.0,21.781250,3.312500,10.625000,1022.968750,2.890625,0.0,3.390625,-24.776109,134.755000
1,new zealand,3.437500,3.453125,3.421875,3.421875,3.562500,3.65625,3.796875,3.890625,4.109375,4.375000,4.515625,4.859375,5.109375,5.359375,5.812500,6.125000,10.718750,13.078125,6.921875,1.406250,-0.812500,-0.421875,0.671875,1.468750,1.578125,1.640625,3.203125,3.453125,4.765625,4.281250,4.250000,2.609375,97.5,7.562500,-9999.0,-9999.0,-9999.0,10.718750,10.171875,8.843750,1027.890625,2.437500,0.0,3.250000,-41.500083,172.834408
2,papua new guinea,4.078125,4.093750,3.687500,3.078125,2.593750,2.40625,2.593750,2.796875,3.140625,3.109375,2.828125,3.156250,3.937500,4.906250,5.046875,5.062500,13.484375,12.171875,9.156250,5.750000,2.203125,0.000000,-2.156250,-3.437500,-3.765625,-3.843750,-3.000000,-2.296875,-1.734375,-1.984375,-1.218750,-0.796875,97.5,3.406250,-9999.0,-9999.0,-9999.0,25.921875,1.984375,10.906250,1011.609375,2.703125,0.0,3.718750,-5.681607,144.248908
3,south africa,3.140625,3.109375,3.234375,3.421875,3.906250,4.15625,4.500000,5.078125,5.718750,6.125000,6.500000,7.109375,8.265625,9.484375,10.265625,10.281250,12.359375,12.109375,10.203125,8.531250,7.671875,7.312500,6.609375,5.078125,5.093750,6.828125,8.531250,10.046875,11.390625,9.140625,6.796875,4.375000,97.5,7.734375,-9999.0,-9999.0,-9999.0,16.937500,3.765625,12.265625,1017.015625,5.500000,0.0,2.828125,-28.816624,24.991639
4,usa,2.234375,2.250000,2.203125,2.468750,2.921875,3.09375,2.937500,3.078125,3.140625,3.343750,3.343750,3.687500,3.906250,4.375000,5.000000,5.250000,9.562500,11.328125,10.656250,9.609375,8.062500,6.703125,5.250000,4.953125,4.937500,5.515625,6.609375,8.734375,9.796875,9.281250,8.984375,2.906250,97.5,5.531250,-9999.0,-9999.0,-9999.0,24.968750,-0.093750,6.968750,1013.718750,2.593750,0.0,2.234375,39.783730,-100.445882


- Cuando ya tengamos todos los datos deseados juntaremos los dos csv.

In [23]:
df_seleccion.head(2)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,country2,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.78373,-100.445882,HN,1,0,0,1,0,0,4,7,1,9.116327
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.78373,-100.445882,HN,1,0,0,1,0,0,3,7,7,3.804860


In [28]:
df_clima_final.rename(columns={"paises":"country"}, inplace=True)

In [29]:
df_final = pd.merge(left = df_seleccion, right= df_clima_final, how= "left", on= "country")
df_final.head()

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud_x,longitud_x,country2,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM,wind_speed950mb,wind_speed900mb,wind_speed850mb,wind_speed800mb,wind_speed750mb,wind_speed700mb,wind_speed650mb,wind_speed600mb,wind_speed550mb,wind_speed500mb,wind_speed450mb,wind_speed400mb,wind_speed350mb,wind_speed300mb,wind_speed250mb,wind_speed200mb,rh950mb,rh900mb,rh850mb,rh800mb,rh750mb,rh700mb,rh650mb,rh600mb,rh550mb,rh500mb,rh450mb,rh400mb,rh350mb,rh300mb,rh250mb,rh200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.speed,latitud_y,longitud_y
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.783730,-100.445882,HN,1,0,0,1,0,0,4,7,1,9.116327,2.234375,2.2500,2.203125,2.46875,2.921875,3.09375,2.93750,3.078125,3.140625,3.343750,3.343750,3.687500,3.90625,4.375000,5.000000,5.250000,9.56250,11.328125,10.65625,9.609375,8.0625,6.703125,5.2500,4.953125,4.93750,5.515625,6.609375,8.734375,9.796875,9.281250,8.984375,2.90625,97.5,5.531250,-9999.0,-9999.0,-9999.0,24.96875,-0.09375,6.96875,1013.71875,2.593750,0.0,2.234375,39.783730,-100.445882
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.783730,-100.445882,HN,1,0,0,1,0,0,3,7,7,3.804860,2.234375,2.2500,2.203125,2.46875,2.921875,3.09375,2.93750,3.078125,3.140625,3.343750,3.343750,3.687500,3.90625,4.375000,5.000000,5.250000,9.56250,11.328125,10.65625,9.609375,8.0625,6.703125,5.2500,4.953125,4.93750,5.515625,6.609375,8.734375,9.796875,9.281250,8.984375,2.90625,97.5,5.531250,-9999.0,-9999.0,-9999.0,24.96875,-0.09375,6.96875,1013.71875,2.593750,0.0,2.234375,39.783730,-100.445882
2,2018,Invalid,usa,48.0,Unespecific,Jun,N,M,39.783730,-100.445882,HN,1,0,0,1,0,0,3,7,3,8.403629,2.234375,2.2500,2.203125,2.46875,2.921875,3.09375,2.93750,3.078125,3.140625,3.343750,3.343750,3.687500,3.90625,4.375000,5.000000,5.250000,9.56250,11.328125,10.65625,9.609375,8.0625,6.703125,5.2500,4.953125,4.93750,5.515625,6.609375,8.734375,9.796875,9.281250,8.984375,2.90625,97.5,5.531250,-9999.0,-9999.0,-9999.0,24.96875,-0.09375,6.96875,1013.71875,2.593750,0.0,2.234375,39.783730,-100.445882
3,2018,Unprovoked,australia,27.0,Unespecific,Jun,N,M,-24.776109,134.755000,HS,1,0,0,1,0,0,3,7,7,6.313925,3.453125,3.1875,3.015625,2.84375,2.812500,2.84375,2.84375,3.171875,3.343750,3.640625,4.265625,4.640625,5.00000,5.390625,5.921875,6.484375,13.71875,11.500000,7.06250,4.140625,2.0625,0.625000,-0.3125,-1.390625,-2.03125,-1.718750,-1.140625,-0.468750,0.328125,2.265625,4.781250,2.56250,97.5,3.515625,-9999.0,-9999.0,-9999.0,21.78125,3.31250,10.62500,1022.96875,2.890625,0.0,3.390625,-24.776109,134.755000
4,2018,Unprovoked,australia,27.0,Unespecific,Jun,N,M,-24.776109,134.755000,HS,1,0,0,1,0,0,3,7,7,6.313925,3.453125,3.1875,3.015625,2.84375,2.812500,2.84375,2.84375,3.171875,3.343750,3.640625,4.265625,4.640625,5.00000,5.390625,5.921875,6.484375,13.71875,11.500000,7.06250,4.140625,2.0625,0.625000,-0.3125,-1.390625,-2.03125,-1.718750,-1.140625,-0.468750,0.328125,2.265625,4.781250,2.56250,97.5,3.515625,-9999.0,-9999.0,-9999.0,21.78125,3.31250,10.62500,1022.96875,2.890625,0.0,3.390625,-24.776109,134.755000


In [30]:
df_final.to_csv("../Ficheros/ETL2.csv")